# FX Market Making Analytics Portfolio

**Author**: Mame Faty Lo  
**Purpose**: Showcase FX market analysis, data cleaning, visualization, and automation for major currency pairs relevant to Swissquote.  

**Objectives**:
- Load and clean FX market data.
- Analyse **mid-price trends**, **bid-ask spreads**, **returns**, and **volatility**. 
- Simulate **market making strategies** and liquidity provision.
- Generate **interactive charts and static visualisations** to support insights.

**Notes**:
- Data can be loaded from bundled sample CSVs or a snapshot pickle for convenience.
- Volume, bid, and ask columns are included to mimic real FX market data.
- All analysis is reproducible and ready for extension to live data sources.

In [7]:
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Default Plotly template and size
pio.templates.default = "plotly_dark"
px.defaults.width = 900
px.defaults.height = 520

# See my custom scripts
from src import downloader_fx
from src import data_loader
from src import data_cleaner

print("[INFO] Scripts imported successfully")

[INFO] Scripts imported successfully


# Download FX Data (1-Minute Bars)

We download 1-minute FX data from **HistData.com** for selected currency pairs.  

- Data will be saved as **one CSV per pair** in `data/raw/fx_full/`.  
- Columns include: `DateTime`, `Open`, `High`, `Low`, `Close`, `Volume`.  
- If the CSVs already exist, this step will skip downloading to save time.  
- We fetch **3 months of historical data** as a snapshot for analysis.


## Load API Keys from Environment Variables

We keep API credentials separate from the notebook for security and reproducibility. This cell loads your OANDA (FX) and Binance (Digital Assets) API keys from the `.env` file located in the `env/` folder at the project root. Keys will be stored in variables for subsequent data fetching.

## FX Market Analysis

### Basic Descriptive Statistics

Quick descriptive statistics for numeric columns (`Open`, `High`, `Low`, `Price`, `Change %`) across all FX pairs. This helps inspect ranges, averages, and variability.

### Price Trends (Normalised)

**Purpose:** Compare relative price movements across FX pairs that have very different scales.

**Method:** We normalise each pair by its price at the first available date:

$$
\text{Normalised Price}_t = \frac{\text{Price}_t}{\text{Price}_{t_0}}
$$

Where:
- $t_0$ is the first available date for the FX pair.
- $\text{Price}_t$ is the price at time $t$.

- All pairs start at 1, enabling direct visual comparison of relative price movements.
- Hover over lines to see exact normalised values and dates.

## Daily Returns and Volatility

**Purpose**:  
Assess the day-to-day price changes and the risk (volatility) associated with each FX pair. This helps understand market behaviour and identify periods of high or low activity.

**Method**:  

1. **Daily Returns**: The percentage change in FX price from one day to the next:  

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} \times 100
$$

Where:  
- \(R_t\) = daily return at day \(t\)  
- \(P_t\) = FX price at day \(t\)  
- \(P_{t-1}\) = FX price at day \(t-1\)  

2. **Volatility**: Standard deviation of daily returns over a period \(n\):  

$$
\text{Volatility}_t = \sqrt{\frac{1}{n-1} \sum_{i=0}^{n-1} (R_{t-i} - \bar{R})^2}
$$

Where:  
- \(R_{t-i}\) = daily return on day \(t-i\)  
- \(\bar{R}\) = mean daily return over \(n\) days  

**Highlight**:  
- Use an interactive Plotly line chart to visualise returns over time.  
- Volatility can be plotted as a rolling standard deviation to show periods of higher risk.  
- Colour-code FX pairs for clarity and hover labels for exact values.

## Daily Returns and Volatility

**Purpose**:  
Assess the day-to-day price changes and the risk (volatility) associated with each FX pair. This helps understand market behaviour and identify periods of high or low activity.

**Method**:  

1. **Daily Returns**: The percentage change in FX price from one day to the next:  

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} \times 100
$$

Where:  
- \(R_t\) = daily return at day \(t\)  
- \(P_t\) = FX price at day \(t\)  
- \(P_{t-1}\) = FX price at day \(t-1\)  

2. **Volatility**: Standard deviation of daily returns over a period \(n\):  

$$
\text{Volatility}_t = \sqrt{\frac{1}{n-1} \sum_{i=0}^{n-1} (R_{t-i} - \bar{R})^2}
$$

Where:  
- \(R_{t-i}\) = daily return on day \(t-i\)  
- \(\bar{R}\) = mean daily return over \(n\) days  

**Highlight**:  
- Use an interactive Plotly line chart to visualise returns over time.  
- Volatility can be plotted as a rolling standard deviation to show periods of higher risk.  
- Colour-code FX pairs for clarity and hover labels for exact values.